In [1]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from torchinfo import summary
from torchvision import datasets, transforms, models

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cuda


## Implementatation

In [2]:
class GroupedConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, n_groups, stride):
        super().__init__()
        self.group_channels = in_channels // n_groups
        self.grouped_conv_paths = nn.ModuleList([
            nn.Conv2d(self.group_channels, out_channels//n_groups, kernel_size=3, stride=stride, padding=1, bias=False) for _ in range(n_groups)
        ])
    
    def forward(self, x):
        x = torch.concat([
            path(x[:, self.group_channels * i:self.group_channels * (i + 1), ...]) for i, path in
            enumerate(self.grouped_conv_paths)], dim=1)
        return x

x_batch = torch.randn(32, 64, 52, 52)
print(GroupedConvolution(64, 128, 32, 1)(x_batch).shape)
print(GroupedConvolution(64, 128, 32, 2)(x_batch).shape)

torch.Size([32, 128, 52, 52])
torch.Size([32, 128, 26, 26])


In [3]:
class DepthwiseSeparableConvolution(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.depthwise_conv = nn.Sequential(
            # GroupedConvolution(in_channels, in_channels, in_channels, stride),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, bias=False, groups=in_channels),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(),
        )
        self.seeparable_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
    def forward(self, x):
        x = self.depthwise_conv(x)
        x = self.seeparable_conv(x)
        return x

x_batch = torch.randn(32, 64, 52, 52)
print(DepthwiseSeparableConvolution(64, 128, 1)(x_batch).shape)
print(DepthwiseSeparableConvolution(64, 128, 2)(x_batch).shape)

torch.Size([32, 128, 52, 52])
torch.Size([32, 128, 26, 26])


In [4]:
class MobileNetV1(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(n_channels, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU()
        )
        self.conv2 = DepthwiseSeparableConvolution(32, 64, 1)
        self.conv3 = DepthwiseSeparableConvolution(64, 128, 2)
        self.conv4 = DepthwiseSeparableConvolution(128, 128, 1)
        self.conv5 = DepthwiseSeparableConvolution(128, 256, 2)
        self.conv6 = DepthwiseSeparableConvolution(256, 256, 1)
        self.conv7 = DepthwiseSeparableConvolution(256, 512, 2)
        self.conv8 = nn.Sequential(*[DepthwiseSeparableConvolution(512, 512, 1) for _ in range(5)])
        self.conv9 = DepthwiseSeparableConvolution(512, 1024, 2)
        self.conv10 = DepthwiseSeparableConvolution(1024, 1024, 1)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, n_classes),
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

In [5]:
mobilenet_v1_model = MobileNetV1(3, 1000)
summary(mobilenet_v1_model, input_size=(1, 3, 224, 224), col_names=['input_size', 'output_size', 'num_params', 'mult_adds'], device='cpu', depth=1)

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Mult-Adds
MobileNetV1                                        [1, 3, 224, 224]          [1, 1000]                 --                        --
├─Sequential: 1-1                                  [1, 3, 224, 224]          [1, 32, 112, 112]         928                       10,838,080
├─DepthwiseSeparableConvolution: 1-2               [1, 32, 112, 112]         [1, 64, 112, 112]         2,528                     29,302,976
├─DepthwiseSeparableConvolution: 1-3               [1, 64, 112, 112]         [1, 128, 56, 56]          9,152                     27,496,832
├─DepthwiseSeparableConvolution: 1-4               [1, 128, 56, 56]          [1, 128, 56, 56]          18,048                    54,993,408
├─DepthwiseSeparableConvolution: 1-5               [1, 128, 56, 56]          [1, 256, 28, 28]          34,688                    26,594,048
├─DepthwiseSeparableConvoluti

## Training

In [6]:
from pathlib import Path

TRAIN_RATIO = 0.8
data_dir = Path('./data/')

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

train_ds = datasets.CIFAR100(data_dir, train=True, download=True, transform=transform)
train_ds, val_ds = random_split(train_ds, (TRAIN_RATIO, 1 - TRAIN_RATIO))
val_ds.transform = transform
test_ds = datasets.CIFAR100(data_dir, train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
import wandb
from src.engine import *

config = dict(batch_size=128, lr=5e-4, epochs=20, dataset='CIFAR100')
with wandb.init(project='pytorch-study', name='MobileNetV1', config=config) as run:
    w_config = run.config
    train_dl = DataLoader(train_ds, batch_size=w_config.batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=w_config.batch_size, shuffle=True)
    
    n_classes = len(train_ds.dataset.classes)
    mobilenet_v1_model = MobileNetV1(3, n_classes).to(DEVICE)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mobilenet_v1_model.parameters(), lr=w_config.lr)
    
    loss_history, acc_history = train(mobilenet_v1_model, train_dl, val_dl, criterion, optimizer, w_config.epochs, DEVICE, run) 

Epoch=20: 100%|██████████| 20/20 [41:45<00:00, 125.27s/it, train_loss=0.141, train_acc=95.87%, val_loss=2.838, val_acc=47.68%]
